# Analysis & saving of individual cells and their trajectories
### Johanna Rahm, Sebastian Malkusch
#### Research group Heilemann
#### Institute for Physical and Theoretical Chemistry, Goethe University Frankfurt a.M.

## Load *.trc files

Choose/insert a directory for the cell files to be searched in (e.g. F:\Marburg\single_colour_tracking\resting). All *.trc  files in the directory will be loaded for the analysis.

In [ ]:
from pySPT.widgets import widgetTrackAnalysis
from pySPT.analysis import coverSlip
from pySPT.analysis import trajectoryStatistics
from pySPT.widgets import widgetLoadHdf5
from pySPT.widgets.widgetNotebooks import init_save_track_analysis
from pySPT.widgets import hdf5
from pySPT.analysis import trackAnalysis
widget_load_hdf5 = widgetLoadHdf5.WidgetLoadHdf5()
widget_track_analysis = widgetTrackAnalysis.WidgetTrackAnalysis()
track_stats = trajectoryStatistics.TrajectoryStatistics()
cover_slip = coverSlip.CoverSlip()
trajectory_stats = trajectoryStatistics.TrajectoryStatistics()
display(widget_track_analysis.dir_box, widget_track_analysis.dir_button)
widget_track_analysis.dir_button.on_click(widget_track_analysis.open_dir)
widget_track_analysis.dir_box.observe(widget_track_analysis.change_dir_box)

Choose/insert the complete directory for the roi.log file which contains the cell sizes (e.g. F:\Marburg\single_colour_tracking\resting\roi.log)

In [ ]:
display(widget_track_analysis.roi_box, widget_track_analysis.roi_button)
widget_track_analysis.roi_button.on_click(widget_track_analysis.open_roi)
widget_track_analysis.roi_box.observe(widget_track_analysis.change_roi_box)

## Parameter for Rossier Fitting

Based on the formular min_track_length * camera_integration_time = n -> tau_threshold = 0.3*n, the tau threshold for diffusion type determination is set. Please insert both values below:

## Run Analysis

In [ ]:
import warnings
warnings.filterwarnings('ignore')
display(widget_track_analysis.run_button)
def run_analysis(event):
    widget_track_analysis.create_clear_output()
    widget_track_analysis.searchSubFolders(widget_track_analysis.dir_name)
    if widget_track_analysis.got_dir and widget_track_analysis.got_roi:
        cover_slip.roi_file = widget_track_analysis.roi_name
        cover_slip.cell_files = widget_track_analysis.file_names        
        cover_slip.create_cells()  
    else:
        widget_track_analysis.warning_file()
    widget_track_analysis.cells = cover_slip.cells
widget_track_analysis.run_button.on_click(run_analysis)

## Plot diffusion histogram

In [ ]:
display(widget_load_hdf5.bin_size_box)

In [ ]:
def plot_diffusions(event):
    widget_load_hdf5.create_clear_output()
    track_stats.cell_sizes = [cell.size for cell in cover_slip.cells]
    track_stats.cell_trajectories = cover_slip.cell_trajectories
    track_stats.run_statistics_no_filter(widget_load_hdf5.bin_size_box.value)
display(widget_load_hdf5.plot_diffusions_button)
widget_load_hdf5.plot_diffusions_button.on_click(plot_diffusions)

## Choose trajectory to plot

Choose a cell. If one wants to plot multiple times, choose a new cell and/or trajectory reload the plotting cell by clicking in the cell and pressing strg+enter.

In [ ]:
def dropdown(event):
    widget_track_analysis.cell_options = widget_track_analysis.get_cell_names(cover_slip.cells)
widget_track_analysis.run_button.on_click(dropdown)
display(widget_track_analysis.drop_down_cells)

Choose a trajectory.

In [ ]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_track_analysis.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            trajectory_indexes = [i+1 for i, _ in enumerate(cover_slip.cell_trajectories[target_cell])]
            widget_track_analysis.trajectory_options = trajectory_indexes
            widget_track_analysis.drop_down_trajectories.options = widget_track_analysis.trajectory_options
widget_track_analysis.drop_down_cells.observe(dropdown)
display(widget_track_analysis.drop_down_trajectories)

In [ ]:
def plot_particle(event):
    widget_track_analysis.create_clear_output()
    cover_slip.plot_trajectory(widget_track_analysis.drop_down_cells.value, widget_track_analysis.drop_down_trajectories.value)
display(widget_track_analysis.plot_button)
widget_track_analysis.plot_button.on_click(plot_particle)

## Save

For each cell a .h5 file will be created, containing all information of the trajectories (unfiltered).

In [ ]:
def save_analysis(event):    
    for cell_index in range(0, len(cover_slip.cells)):
        h5 = hdf5.Hdf5()
        track_analysis = trackAnalysis.TrackAnalysis()
        init_save_track_analysis(h5, cover_slip, cell_index, track_analysis)
display(widget_track_analysis.save_button)
widget_track_analysis.save_button.on_click(save_analysis)